In [50]:
from bs4 import BeautifulSoup
import pandas as pd
import urllib
from urllib.request import urlopen
import re
from tqdm import tqdm
import pickle
import concurrent
import numpy as np
import time
from pymongo import MongoClient

In [41]:
client = MongoClient()
db = client.voice

In [42]:
first_stage = list(db.word.find())
ids = [i["_id"] for i in first_stage]

In [43]:
authors = set()

for i in first_stage:
    for j in i["pronounces"]:
        authors.add(j["author"])

In [44]:
def get_user(user):
    url = "https://forvo.com/user/" + user
    html = urlopen(url).read().decode()
    root = BeautifulSoup(html, "html.parser")
    root = root.find("article", {"class": "user_info"})

    answer = {}
    for i in root.findAll("li"):
        if "Gender" in i.text:
            answer["gender"] = i.find("strong").text

        elif "Accent/country" in i.text:
            answer["accent"] = i.find("strong").text

        elif "Pronunciations" in i.text:
            answer["pronunciations"] = i.text

        elif "Visits" in i.text:
            answer["visits"] = i.find("strong").text

        elif "Position by added words" in i.text:
            answer["word_rank"] = i.find("strong").text

        elif "Position by pronunciations" in i.text:
            answer["pronun_rank"] = i.find("strong").text

    answer["user"] = user
    db.authors.insert_one(answer)
    

In [47]:
for author in tqdm(authors):
    try:
        get_user(author)
    except AttributeError:
        pass

100%|██████████| 4456/4456 [13:26<00:00,  6.08it/s]


In [ ]:
while True:
    for obj in list(db.word.find({'_id': { '$nin': ids}})):
        ids.append(obj["_id"])
        for pronounce in obj["pronounces"]:
            author = pronounce["author"]
            
            if author not in authors:
                authors.add(author)
                try:
                    get_user(author)
                except AttributeError:
                    pass
                print("\r", "New author " + author, end = "")
    time.sleep(5)

 New author dianada